In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [2]:
df_train = pd.read_csv("/home/ajeet/Sourav/train_data.csv")
df_train.head()

,Unnamed: 0,subject_ID,text,Anorexia
0,0,subject1074_1,[pc] [h] 4 keys maybe 5? [w] endo so why doe...,0
1,1,subject1101_1,Because he saw a movie once? It's Canadian ...,0
2,2,subject1120_1,What book should I read next that will scare ...,0
3,3,subject1137_1,''I only drink soda at parties'' Are you ex...,0
4,4,subject1143_1,"Once I named myself 'pros solo', the number ...",0


In [3]:
x = df_train['text']
y = df_train['Anorexia']

In [5]:
from preprocessing import clean_text

for i in range(0, len(x)):
    x[i] = clean_text(str(x[i]))

/tmp/ipykernel_272388/1766723512.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = clean_text(str(x[i]))
/tmp/ipykernel_272388/1766723512.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = clean_text(str(x[i]))
/tmp/ipykernel_272388/1766723512.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = clean_text(str(x[i]))
/tmp/ipykernel_272388/1766723512.py:4: SettingWithCopyWarning: 
A value is trying to be set on a 

In [7]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [8]:
tfidf_vectorizer = TfidfVectorizer()

In [9]:
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [10]:
svm_classifier = SVC(kernel='linear')

In [11]:
svm_classifier.fit(X_train_tfidf, y_train)

SVC(kernel='linear')

In [13]:
y_pred = svm_classifier.predict(X_test_tfidf)

In [14]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(report)

Accuracy: 0.93
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.99      0.96       417
           1       0.83      0.40      0.54        50

    accuracy                           0.93       467
   macro avg       0.88      0.70      0.75       467
weighted avg       0.92      0.93      0.92       467



# Testing

In [30]:
testing_dataset = pd.read_csv("/home/ajeet/Sourav/test_data.csv")
testing_dataset.head()

,Unnamed: 0,subject_ID,num_writings,text,Anorexia
0,0,1152_1,4,Student at uw. Need a few hours away from cam...,1
1,1,1169_1,14,"Haha, nailed it dude.\n Rings are defi...",1
2,2,1441_1,55,"Right, which is why I said those people wo...",1
3,3,1518_1,31,I just have a lot of self hatred and depri...,1
4,4,1773_1,1,Thank you for your long reply! I have been...,1


In [31]:
d = {}
with open("/home/ajeet/Sourav/risk-golden-truth-test.txt", 'r') as f:
    for line in f:
        subject, label = line.split()
        d[subject[7:]] = int(label)

len(d)

320

In [32]:
dataset_dict = dict(zip(testing_dataset['subject_ID'], zip(testing_dataset['text'], testing_dataset['num_writings'], testing_dataset['Anorexia'])))

In [34]:
test_text, test_num_writing, test_label = [], [], []
for key in d.keys():
    text, num_writing, label = [], [], []
    for i in range(1, 11):
        subject_id = key + '_' + str(i)
        p = dataset_dict[subject_id]
        text.append(str(p[0]) + 'the')
        num_writing.append(p[1])
        label.append(p[2])
    test_text.append(text)
    test_num_writing.append(num_writing)
    test_label.append(label)


In [24]:
test_text[1], test_num_writing[1], test_label[1]

(['    the only time ive even heard a teacher deny religion  so blatantly is in "god\'s not dead"     As a Christian, I totally agree that these bland movies need to spread love and not judgement in order to serve God.     Oh granny, you always were a bigot 3 3     i can just imagine granny calling out the "illuminati\'s shenanigans."     Not if your on facebook you\'re not.  fundamentalist blogger says "feminists love violent muslim men"       oh but guys were totally real tho lol     Yeah because rap like Kendrick Lamar and Lupe Fiasco will NEVER be as thoughtful as Limp Bizkit\n\n..???     I kinda agreed with her until i started reading.\n\nSwearing isn\'t an argument.     "Exotic" aka not white     sound like regular anime dialogue to me.\n\nThat being said, still cringey.  my otherkin reflection       Even if it was true, it\'s not like dogs can understand you.     (they were black)\n\nthanks for that, truly integral to the post.     at least he\'s right about the last part.  my d

In [35]:
import numpy as np
from tqdm import tqdm

def evaluate(test_dataset, model):
    probs, target, delays = [], [], []
    for features_text, delay, label in tqdm(test_dataset):
        text = tfidf_vectorizer.transform(features_text)
        temp = svm_classifier.predict(text)
        target.append(label[0])
        probs.append(temp)
        delays.append(delay)

    probs = np.array(probs)
    target = np.array(target)
    return probs, target, delays

In [36]:
probs, label, delay = evaluate(zip(test_text, test_num_writing, test_label), svm_classifier)

320it [00:11, 27.25it/s]


In [37]:
import math

def get_delays(preds, delays_all):
    delays = []
    for i in range(0, len(preds)):
        delay = 0
        for j in range(0, len(preds[i])):
            if preds[i][j] == 1:
                # delay += delays_all[i][j]
                break
            delay += delays_all[i][j]
        delays.append(delay)
    return delays

import numpy as np

def lco(k, o):
    if k - o > 700:
        return 1.0
    elif k - o < -700:
        return 0.0
    else:
        return 1 - (1 / (1 + np.exp(k - o)))


def calculate_ERDE(preds, delays, ground_truth, cfn, ctp, o):
    cfp = np.sum(ground_truth)/len(ground_truth)
    erde_values = []
    for pred, gt, delay in zip(preds, ground_truth, delays):
        if pred == 1 and gt == 1:
            erde = lco(delay, o) * ctp 
        elif pred == 0 and gt == 1:
            erde = cfn
        elif pred == 1 and gt == 0:
            erde = cfp
        else:
            erde = 0
        erde_values.append(erde)
    return sum(erde_values) / len(erde_values)

In [42]:
from sklearn.metrics import f1_score, precision_score, recall_score

cfp = 0.01  # Cost of false positive
cfn = 1     # Cost of false negative
ctp = 1     # Cost of true positive
o = 5 

delays = get_delays(probs, delay)


preds = []
for i in probs:
    if sum(i) > 0: 
        preds.append(1)
    else: 
        preds.append(0)

print('f1 = ',f1_score(label, preds))
print('precisoin = ', precision_score(label, preds))
print('recall = ', recall_score(label, preds))


print('ERDE5 = ', 100*calculate_ERDE(preds, delays, label, cfn, ctp, o))
print('ERDE50 = ', 100*calculate_ERDE(preds, delays, label, cfn, ctp, o*10))

f1 =  0.7631578947368421
precisoin =  0.8285714285714286
recall =  0.7073170731707317
ERDE5 =  8.728262280447197
ERDE50 =  5.827983461871313
